## 1.1 Detecting and Fixing Errors in dirty_data.csv 

General approach
Look for potential dirty data in the following areas:
- integrity constraints
- data entry error
- wrong categorical data
- violation of referential integrity
- duplicated data
- go against value range
- wrong encoding
- wrong representations
- wrong names and numbers

Import necessary libraries

In [91]:
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [140]:
dirty_data_path = "data\Group109_dirty_data.csv"
missing_data_path = "data\Group109_missing_data.csv"
branch_path = r"data\branches.csv"
edges_path = "data\edges.csv"
nodes_path = r"data\nodes.csv"

dirty_df = pd.read_csv(dirty_data_path)
missing_df = pd.read_csv(missing_data_path)
branch_df = pd.read_csv(branch_path)
edges_df = pd.read_csv(edges_path)
nodes_df = pd.read_csv(nodes_path)

In [53]:
branch_df

,branch_code,branch_name,branch_lat,branch_lon
0,NS,Nickolson,-37.773803,144.983647
1,TP,Thompson,-37.861835,144.905716
2,BK,Bakers,-37.815834,145.046450


In [54]:
dirty_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   order_id                 500 non-null    object 
 1   date                     500 non-null    object 
 2   time                     500 non-null    object 
 3   order_type               500 non-null    object 
 4   branch_code              500 non-null    object 
 5   order_items              500 non-null    object 
 6   order_price              500 non-null    float64
 7   customer_lat             500 non-null    float64
 8   customer_lon             500 non-null    float64
 9   customerHasloyalty?      500 non-null    int64  
 10  distance_to_customer_KM  500 non-null    float64
 11  delivery_fee             500 non-null    float64
dtypes: float64(5), int64(1), object(6)
memory usage: 47.0+ KB


In [55]:
dirty_df.describe()

,order_price,customer_lat,customer_lon,customerHasloyalty?,distance_to_customer_KM,delivery_fee
count,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000
mean,480.712900,-30.753946,143.504403,0.102000,8.629274,13.877162
std,254.034843,25.337436,16.299630,0.302951,1.596279,2.378285
min,31.750000,-37.827188,-37.822231,0.000000,3.613000,5.646222
25%,300.625000,-37.818738,144.952786,0.000000,7.750500,12.660927
50%,434.250000,-37.811755,144.963914,0.000000,8.639500,13.849738
75%,633.250000,-37.804505,144.980037,0.000000,9.633500,15.229668
max,1361.500000,145.005221,145.015449,1.000000,13.735000,20.088572


In [56]:
dirty_df.head()

,order_id,date,time,order_type,branch_code,order_items,order_price,customer_lat,customer_lon,customerHasloyalty?,distance_to_customer_KM,delivery_fee
0,ORDX02948,2018-03-11,10:11:49,Breakfast,BK,"[('Cereal', 4), ('Eggs', 3), ('Coffee', 1), ('...",206.00,-37.805040,144.963243,0,7.615,14.403915
1,ORDC07988,2018-03-06,10:01:41,Breakfast,NS,"[('Coffee', 2), ('Cereal', 5), ('Eggs', 10), (...",437.00,-37.815989,144.983435,0,8.914,13.807773
2,ORDI00568,2018-07-05,14:05:04,Lunch,NS,"[('Fries', 6), ('Chicken', 5), ('Salad', 7), (...",507.40,-37.806281,144.941960,1,9.316,15.028384
3,ORDI06756,2018-04-23,11:43:05,Breakfast,NS,"[('Coffee', 3), ('Pancake', 6), ('Eggs', 3)]",234.00,-37.822259,144.946977,0,9.975,15.217270
4,ORDX01986,2018-04-29,11:53:14,Breakfast,BK,"[('Coffee', 6), ('Cereal', 1), ('Eggs', 2), ('...",134.25,-37.815950,144.986001,0,6.038,13.677500


As stated in the assignment brief, we will not be looking for any fixing values within the following columns as there are no errors in dirty data for them:
- `order_id`
- `time`
- the numeric quantity in `order_items`
- `delivery_fee`

- [x] check if dates, time is properly formatted
- [x] check if branch code is correct
- [ ] check if order price makes sense according to number of items ordered
- [ ] check if distance to customer is calculated properly in shortest distance
- [ ] check if customer_lat and customer_lon exist in nodes
- [x] check if order_type is correct according to time
- [ ] check if customer does / does not have loyalty according to delivery fee ()


Delivery fee is calculated using a different method for each branch.
The fee depends linearly (but in different ways for each branch) on:
a. weekend or weekday (1 or 0) - as a continuous variable
b. time of the day (morning 0, afternoon 1, evening 2) - as a continuous variable
c. distance between branch and customer


In [57]:
dirty_df['branch_code'].value_counts()

TP    169
NS    163
BK    144
ns     11
tp      7
bk      6
Name: branch_code, dtype: int64

From the output above, we can see that some of the branch codes have been inputted in the wrong representation. Instead of being inputted in all uppercase, some of the values are shown as lowercase. We will need to perform data transformation on this column to convert all values to uppercase letters

In [58]:
dirty_df['branch_code'] = dirty_df['branch_code'].apply(str.upper)

According to the assignment brief, all string date values in column `date` should be in the format YYYY-MM-DD. We can verify if this is the case by using `pd.to_datetime` function on the `date` column and see if all date values fit the format `%Y-%m-%d`

In [59]:
try:
    pd.to_datetime(dirty_df['date'], format='%Y-%m-%d', errors='raise')
except ValueError as e:
    print("Error occurred. Unable to convert the following date:")
    print(e)
else:
    print('No error')

Error occurred. Unable to convert the following date:
time data 06-10-2018 doesn't match format specified


As we can see, not all the date values are in the correct format. Therefore we will have to iterate through each date value and determine which one of the follow formats the date value can be in:
1. YYYY-MM-DD
2. DD-MM-YYYY
3. YYYY-DD-MM

In [60]:
for index, row in dirty_df.iterrows():
    new_row = pd.Series(row)

    # Check if date fits the format YYYY-MM-DD
    try:
        new_row = pd.to_datetime(row['date'], format='%Y-%m-%d')
    except:
        # If not, check if it fits the format YYYY-DD-MM
        try: 
            new_row = pd.to_datetime(row['date'], format='%Y-%d-%m')
        # Else, check if it fits the format DD-MM-YYYY
        except:
            new_row = pd.to_datetime(row['date'], format='%d-%m-%Y')
    dirty_df.at[index, 'date'] = new_row

dirty_df['date'] = pd.to_datetime(dirty_df['date'], format='%Y-%m-%d').dt.strftime('%Y-%m-%d')

Now we evaluate the `order_type` column. Since we know for certain that the time column does not have any errors in it, we can verify if the `order_type` is correct according to the time. The order should be the following according to the times:
- 08:00:00 - 12:00:00 = Breakfast
- 12:00:01 - 16:00:00 = Lunch
- 16:00:01 - 20:00:00 = Dinner

In [61]:
dirty_df['order_type'].value_counts()

Breakfast    170
Lunch        165
Dinner       165
Name: order_type, dtype: int64

To check if the correct order type has been inputted, we create the function `find_order-type`. It looks through each string time value in `time` column, converts this string into datetime format. Then we check the timestamp to see if it fits within the Breakfast, Lunch or Dinner time slots

In [62]:
def find_order_type(time):
    timestamp = pd.to_datetime(time, format='%H:%M:%S').time()
    if timestamp >= pd.to_datetime('08:00:00').time() and timestamp <= pd.to_datetime('12:00:00').time():
        return 'Breakfast'
    elif timestamp >= pd.to_datetime('12:00:01').time() and timestamp < pd.to_datetime('16:00:00').time():
        return 'Lunch'
    elif timestamp >= pd.to_datetime('16:00:01').time() and timestamp <= pd.to_datetime('20:00:00').time():
        return 'Dinner'
    else:
        return 'Error'

In [63]:
# output values where calculated order type does not match the order type in the dataset
dirty_df[dirty_df['time'].apply(find_order_type) != dirty_df['order_type']].head()

,order_id,date,time,order_type,branch_code,order_items,order_price,customer_lat,customer_lon,customerHasloyalty?,distance_to_customer_KM,delivery_fee
14,ORDC09610,2018-10-06,17:07:36,Lunch,NS,"[('Fish&Chips', 5), ('Shrimp', 2), ('Pasta', 4)]",393.0,-37.826028,144.984514,0,9.578,17.378357
19,ORDC06273,2018-09-05,16:06:45,Lunch,NS,"[('Pasta', 5), ('Shrimp', 10), ('Fish&Chips', ...",1361.5,-37.814936,144.927351,0,10.813,16.896152
20,ORDB10659,2018-04-20,11:32:57,Dinner,TP,"[('Eggs', 7), ('Coffee', 10), ('Cereal', 6), (...",597.5,-37.818900,144.952797,0,8.576,11.161592
37,ORDK01676,2018-03-16,10:21:58,Dinner,BK,"[('Eggs', 2), ('Pancake', 2)]",92.5,-37.801158,144.957692,0,8.326,13.278789
54,ORDB10190,2018-10-08,17:17:44,Breakfast,TP,"[('Fish&Chips', 2), ('Salmon', 2)]",152.0,-37.813657,144.957285,0,8.419,13.088105


As shown above, there are orders in which order type has been incorrectly inputted according to the time. We can fix this issue by performing the following

In [64]:
dirty_df['order_type'] = dirty_df['time'].apply(find_order_type)

In [65]:
dirty_df['order_type'].value_counts()

Dinner       170
Breakfast    166
Lunch        164
Name: order_type, dtype: int64

We determine if the branch and customer locations in `dirty_df` can be accurately found in the provided Graph in `nodes.csv`

In [66]:
# check if customer node exists
def check_customer_nodes(row):
    customer_node = nodes_df[(nodes_df['lat'] == row['customer_lat']) & (nodes_df['lon'] == row['customer_lon'])]

    if customer_node.empty:
        return True
    else:
        return False

# check if branch node exists
def check_branch_nodes(row):
    branch_lat = branch_df[branch_df['branch_code'] == row['branch_code']]['branch_lat'].values[0]
    branch_lon = branch_df[branch_df['branch_code'] == row['branch_code']]['branch_lon'].values[0]

    branch_node = nodes_df[(nodes_df['lat'] == branch_lat) & (nodes_df['lon'] == branch_lon)]

    if branch_node.empty:
        return True 
    else:
        return False

In [67]:
# Retrieve the number of rows where the customer node does not exist in the nodes.csv file
dirty_df.apply(check_branch_nodes, axis=1).sum()

0

From the above results, we can see that all the branch nodes are accounted for and each node exists in the provided graph

In [68]:
dirty_df.apply(check_customer_nodes, axis=1).sum()

41

When checking for customer nodes, it is evident that there are 41 instances of orders where we are unable to identify the node in the graph according to the provided customer longitude and latitude values. To investigate further into the cause of this issue, let's output the first 15 instances where this is the case.

In [69]:
dirty_df[dirty_df.apply(check_customer_nodes, axis=1)][['customer_lat', 'customer_lon']].head(15)

,customer_lat,customer_lon
5,37.814994,144.960538
7,37.816395,144.938170
12,37.816482,144.964894
25,37.824000,144.953766
35,37.811124,145.001788
38,37.799866,145.002800
45,37.814037,144.985480
49,37.813155,144.968360
52,37.804832,144.950241
53,37.812135,144.962341


From the outputted data above, we can see 2 prominent errors within the data:
1. <b>There are some instances in which customer latitude has a missing negative symbol at the front. </b>

Take the row at index 5 for example where latitude and longitude are 37.814994 and 144.960538 respectively. If we look at the `nodes.csv` file:

In [70]:
index = 5
# modify latitude by multiplying it by -1
lat = dirty_df.loc[index,]['customer_lat'] * -1
lon = dirty_df.loc[index,]['customer_lon']

# check if the modified latitude exists in the nodes.csv file
nodes_df[(nodes_df['lat'] == lat) & (nodes_df['lon'] == lon)]

,node,lat,lon
3793,6167489464,-37.814994,144.960538


As shown above, the following node exists in the graph when the latitude value is included with a negative value.


2. <b>There are some instance in which customer longitude and customer latitude are placed incorrectly and have been swapped with one another</b>

Take row at index 135 for example where latitude and longitude are 145.005221 and -37.817570 respectively. It is evident that these have been inputted incorrectly and the values have swapped over.

In [71]:
index = 135
# modify latitude by multiplying it by -1
lat = dirty_df.loc[index,]['customer_lon']
lon = dirty_df.loc[index,]['customer_lat']

# check if the modified latitude exists in the nodes.csv file
nodes_df[(nodes_df['lat'] == lat) & (nodes_df['lon'] == lon)]

,node,lat,lon
9191,1463620803,-37.81757,145.005221


As shown above, the following node exists in the graph when the customer latitude and longitude values are swapped over.

We fix the issue with customer longitude and latitude values using the following custom function `find_customer_node` which first tries to find the node of the customer based on the given latitude and longitude values. If it is unable to do so, we try the following approaches next:
1. Multiply the latitude value by negative and check to see if these coordinates exist in `nodes_df`. Else;
2. Swap the longitude and latitude values and check to see if these coordinates exist in `nodes_df`

If none of these approaches allow us to find a node, we will raise a `ValueError`

In [72]:
def calc_customer_node(row):
    cus_lat = row['customer_lat']
    cus_lon = row['customer_lon']
    customer_node = nodes_df[(nodes_df['lat'] == cus_lat) & (nodes_df['lon'] == cus_lon)]

    # If the customer node does not exist, check for misinput of latitude and longitude values
    if customer_node.empty:
        # multiply the latitude by -1 
        customer_node = nodes_df[(nodes_df['lat'] == -cus_lat) & (nodes_df['lon'] == cus_lon)]
        # If the customer node still does not exist, try swapping the latitude and longitude values
        if customer_node.empty:
            customer_node = nodes_df[(nodes_df['lat'] == cus_lon) & (nodes_df['lon'] == cus_lat)]

    # If the customer node still does not exist, raise an error
    if customer_node.empty:
        raise ValueError("Customer node does not exist in the nodes.csv file")
    
    return customer_node.iloc[0]['lat'], customer_node.iloc[0]['lon']


In [73]:
dirty_df['customer_coords'] = dirty_df.apply(calc_customer_node, axis=1)
dirty_df[['customer_lat', 'customer_lon']] = pd.DataFrame(dirty_df['customer_coords'].to_list(), index=dirty_df.index)

dirty_df.drop(columns=['customer_coords'], inplace=True)

In [74]:
dirty_df.apply(check_customer_nodes, axis=1).sum()

0

When we run `check_customer_nodes` again we can see that all of the coordinates can be accounted for and found in the `nodes.csv` file

Now we check if `distance_to_customer_KM` is correct. To start with this, we utilise `Graph()` function from `networkx` library to construct a Graph based on the nodes, edges provided by the Assignment brief in `nodes.csv` and `edges.csv` respectively

In [75]:
G = nx.Graph()
G.add_nodes_from(nodes_df['node'])
for index, row in edges_df.iterrows():
    G.add_edge(row['u'], row['v'], weight=row['distance(m)'])

In [76]:
def find_branch_node(row):
    branch_lat = branch_df[branch_df['branch_code'] == row['branch_code']]['branch_lat'].values[0]
    branch_lon = branch_df[branch_df['branch_code'] == row['branch_code']]['branch_lon'].values[0]

    branch_node = nodes_df[(nodes_df['lat'] == branch_lat) & (nodes_df['lon'] == branch_lon)]

    return branch_node

def find_shortest_path(row):
    cus_lat = row['customer_lat']
    cus_lon = row['customer_lon']
    customer_node = nodes_df[(nodes_df['lat'] == cus_lat) & (nodes_df['lon'] == cus_lon)]
    
    branch_node = find_branch_node(row)

    # Find the shortest path between the customer node and the branch node
    try:
        # calculates the shortest path using djikstra's algorithm in M
        shortest_path = nx.shortest_path_length(G, source=customer_node['node'].values[0], target=branch_node['node'].values[0], weight='weight')

        # calculate to KM
        shortest_path = shortest_path/1000
    except nx.NetworkXNoPath:
        raise ValueError("No path exists between the customer node and the branch node")
    
    return shortest_path

In [77]:
dirty_df['shortest_path'] = dirty_df.apply(find_shortest_path, axis=1)

In [78]:
dirty_df[dirty_df['distance_to_customer_KM'] != dirty_df['shortest_path']][['distance_to_customer_KM', 'shortest_path']].head()

,distance_to_customer_KM,shortest_path
1,8.914,8.624
11,8.330,7.699
15,8.667,7.938
18,8.245,8.787
22,5.435,11.912


Once we have correctly calculated the shortest distance to the customer according to Djikstra's algorithm, we replace the data in `distance_to_customer_KM` column with the correct distance data in `shortest_path` column

In [79]:
dirty_df['distance_to_customer_KM'] = dirty_df['shortest_path']
dirty_df.drop(columns=['shortest_path'], inplace=True)

Check if  `order_price` is correct

In [80]:
dirty_df['order_items'] = dirty_df['order_items'].apply(eval)

Run this code to determine all the menu items provided

In [87]:
dirty_df.explode('order_items')['order_items'].apply(lambda x: x[0]).unique().tolist()

['Cereal',
 'Eggs',
 'Coffee',
 'Pancake',
 'Fries',
 'Chicken',
 'Salad',
 'Burger',
 'Salmon',
 'Shrimp',
 'Fish&Chips',
 'Pasta',
 'Steak']

In [163]:
MENU_ITEMS = ['Cereal',
 'Eggs',
 'Coffee',
 'Pancake',
 'Fries',
 'Chicken',
 'Salad',
 'Burger',
 'Salmon',
 'Shrimp',
 'Fish&Chips',
 'Pasta',
 'Steak']

def create_order_dict(row):
    order_mapping = {item: index for index, item in enumerate(MENU_ITEMS)}
    order = [0 for _ in MENU_ITEMS]
    for item in row['order_items']:
        order_index = order_mapping[item[0]]
        order[order_index] += item[1]
    return order


def sum_order_items(row):
    res = 0
    for item in row['order_items']:
        res += 1
    return res

We can use the clean data in missing_data.csv to determine the correct menu item price

In [144]:
missing_df['order_items'] = missing_df['order_items'].apply(eval)

In [153]:
missing_df['order_price'].isna().sum()

0

In [169]:
missing_df[missing_df.apply(sum_order_items, axis=1) == 2]

,order_id,date,time,order_type,branch_code,order_items,order_price,customer_lat,customer_lon,customerHasloyalty?,distance_to_customer_KM,delivery_fee
7,ORDA00394,2018-06-12,13:24:30,Lunch,BK,"[(Salad, 7), (Burger, 1)]",151.40,-37.815136,144.941916,0,9.544,NaN
12,ORDY08471,2018-06-17,13:34:38,Lunch,TP,"[(Chicken, 1), (Steak, 8)]",392.00,-37.810195,144.997125,0,10.784,14.924616
14,ORDX03398,2018-04-28,11:53:14,Breakfast,NaN,"[(Eggs, 9), (Coffee, 8)]",258.00,-37.815691,144.984638,0,6.029,13.376741
20,ORDZ07062,2018-07-10,14:15:12,Lunch,NS,"[(Salad, 9), (Fries, 8)]",250.80,-37.824557,144.984639,0,9.481,14.422345
23,ORDJ00566,2018-12-09,19:19:26,Dinner,TP,"[(Fish&Chips, 4), (Salmon, 2)]",222.00,-37.819902,144.990693,0,9.797,15.607731
...,...,...,...,...,...,...,...,...,...,...,...,...
483,ORDK06710,2018-06-07,13:14:21,Lunch,BK,"[(Salad, 8), (Steak, 1)]",182.60,-37.803646,144.949025,0,8.877,15.371713
486,ORDJ10411,2018-07-14,14:25:21,Lunch,TP,"[(Fries, 1), (Chicken, 8)]",268.00,-37.816218,145.007396,0,11.490,15.953062
491,ORDB00037,2018-06-07,13:14:21,Lunch,TP,"[(Chicken, 6), (Steak, 1)]",237.00,-37.805194,144.941074,0,10.407,NaN
493,ORDX06982,2018-01-13,08:20:16,Breakfast,NaN,"[(Pancake, 1), (Cereal, 9)]",213.25,-37.816493,144.986020,1,5.953,6.263917


In [170]:
a = np.array(missing_df.apply(create_order_dict, axis=1).tolist())
b = np.array(missing_df['order_price'].tolist())

In [139]:
a = np.array(dirty_df.apply(create_order_dict, axis=1).tolist())
b = np.array(dirty_df['order_price'].tolist())

# Solve the system of equations using least squares regression
solution, residuals, _, _ = np.linalg.lstsq(a, b, rcond=None)

solution

array([19.8346994 , 21.60767191, 11.9113183 , 23.67133084, 15.75167966,
       27.15715224, 16.09090613, 31.70140819, 38.93932756, 51.93875105,
       34.26621038, 30.19122647, 46.19117906])

In [136]:
import numpy as np

# Assuming your coefficient matrix is named 'coefficients' and your constant vector is named 'constants'
coefficients = np.random.rand(500, 13)  # Example coefficients, replace this with your actual coefficients
constants = np.random.rand(500)         # Example constants, replace this with your actual constants

# Solve the system of equations
solution = np.linalg.solve(coefficients, constants)

print("Solution shape:", solution.shape)


LinAlgError: Last 2 dimensions of the array must be square

In [137]:
import numpy as np

# Assuming your coefficient matrix is named 'coefficients' and your constant vector is named 'constants'
coefficients = np.random.rand(500, 13)  # Example coefficients, replace this with your actual coefficients
constants = np.random.rand(500)         # Example constants, replace this with your actual constants

# Solve the system of equations using least squares regression
solution, residuals, _, _ = np.linalg.lstsq(coefficients, constants, rcond=None)

print("Solution shape:", solution.shape)


Solution shape: (13,)


In [138]:
solution

array([ 0.01535877,  0.02646134,  0.16947561,  0.06515357,  0.05740383,
        0.09058637,  0.06181313, -0.04802618,  0.07791322,  0.19966148,
        0.07772996,  0.10182666,  0.0641936 ])

In [133]:
dirty_df

,order_id,date,time,order_type,branch_code,order_items,order_price,customer_lat,customer_lon,customerHasloyalty?,distance_to_customer_KM,delivery_fee
0,ORDX02948,2018-03-11,10:11:49,Breakfast,BK,"[(Cereal, 4), (Eggs, 3), (Coffee, 1), (Pancake...",206.00,-37.805040,144.963243,0,7.615,14.403915
1,ORDC07988,2018-03-06,10:01:41,Breakfast,NS,"[(Coffee, 2), (Cereal, 5), (Eggs, 10), (Pancak...",437.00,-37.815989,144.983435,0,8.624,13.807773
2,ORDI00568,2018-07-05,14:05:04,Lunch,NS,"[(Fries, 6), (Chicken, 5), (Salad, 7), (Burger...",507.40,-37.806281,144.941960,1,9.316,15.028384
3,ORDI06756,2018-04-23,11:43:05,Breakfast,NS,"[(Coffee, 3), (Pancake, 6), (Eggs, 3)]",234.00,-37.822259,144.946977,0,9.975,15.217270
4,ORDX01986,2018-04-29,11:53:14,Breakfast,BK,"[(Coffee, 6), (Cereal, 1), (Eggs, 2), (Pancake...",134.25,-37.815950,144.986001,0,6.038,13.677500
...,...,...,...,...,...,...,...,...,...,...,...,...
495,ORDZ08211,2018-03-27,10:42:15,Breakfast,NS,"[(Coffee, 3), (Pancake, 6), (Cereal, 1), (Eggs...",409.00,-37.809194,144.973066,0,7.240,12.114681
496,ORDA06488,2018-09-23,16:47:19,Dinner,BK,"[(Salmon, 1), (Pasta, 7), (Shrimp, 6), (Fish&C...",802.50,-37.805559,144.945085,0,9.100,19.245383
497,ORDY05487,2018-07-06,14:05:04,Lunch,TP,"[(Burger, 3), (Steak, 7)]",408.00,-37.811246,145.000438,0,11.050,14.272925
498,ORDB04585,2018-09-21,16:47:19,Dinner,TP,"[(Pasta, 1), (Shrimp, 9), (Fish&Chips, 10), (S...",945.50,-37.820320,144.957726,1,7.933,12.252722


In [114]:
b.shape

(500,)

In [118]:
import numpy as np

# Coefficients of the equations
coefficients = np.array([[2, 1, -1],
                         [-3, -1, 2],
                         [-2, 1, 2]])

# Constants on the right-hand side of the equations
constants = np.array([8, -11, -3])

# Solve the system of equations
solution = np.linalg.solve(coefficients, constants)

print("Solution:", solution)


Solution: [ 2.  3. -1.]


In [120]:
a.shape, b.shape

((2, 3), (2,))

Check if `customerHasloyalty?` is correct

In [50]:
from sympy import symbols, solve

In [ ]:
def calc_customer_loyalty(row):
    

In [6]:
dirty_df['order_items'][0]

"[('Cereal', 4), ('Eggs', 3), ('Coffee', 1), ('Pancake', 2)]"

In [7]:
dirty_df['order_price'][0]

206.0

Check if `order_price` is correct

In [48]:
dirty_df[['order_items', 'order_price']].head()

,order_items,order_price
0,"[('Cereal', 4), ('Eggs', 3), ('Coffee', 1), ('...",206.00
1,"[('Coffee', 2), ('Cereal', 5), ('Eggs', 10), (...",437.00
2,"[('Fries', 6), ('Chicken', 5), ('Salad', 7), (...",507.40
3,"[('Coffee', 3), ('Pancake', 6), ('Eggs', 3)]",234.00
4,"[('Coffee', 6), ('Cereal', 1), ('Eggs', 2), ('...",134.25


## 1.2 Imputating data